In [18]:
from plyer import notification
from datetime import datetime
import pandas as pd
import requests
import sqlite3

In [19]:
def alerta(nivel,base, etapa):
    
    if nivel == 1:
        nivel = 'nivel baixo'
    elif nivel == 2:
        nivel = 'nível médio'
    else:
        nivel = 'nível alto'
    
    data_= datetime.now().strftime('%d/%m/%Y %H:%M:%S.%f')

    notification.notify (
    title = f'ATENÇÃO: Alerta {nivel}',
    message = f'Falha no carregamento da base {base} na etapa {etapa}\n{data_}',
    )

In [20]:
def load_estados():
    url = 'https://brasilapi.com.br/api/ibge/uf/v1'
    req_Estados = requests.get(url)
    if req_Estados.status_code == 200:
        dados_Estados = pd.DataFrame(req_Estados.json())
        return(dados_Estados)
    else:
        alerta('3', 'estados do Brasil', 'carregamento de dados')
        return
df_brasil = load_estados()
df_brasil.head()

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


In [21]:
df_brasil = df_brasil.rename(columns={'id':'ID', 'sigla': 'Sigla', 'nome': 'Nome', 'regiao': 'Regiao'})
df_brasil['Regiao'] = [regiao['nome'] for regiao in df_brasil['Regiao']]
df_brasil

,ID,Sigla,Nome,Regiao
0,11,RO,Rondônia,Norte
1,12,AC,Acre,Norte
2,13,AM,Amazonas,Norte
3,14,RR,Roraima,Norte
4,15,PA,Pará,Norte
5,16,AP,Amapá,Norte
6,17,TO,Tocantins,Norte
7,21,MA,Maranhão,Nordeste
8,22,PI,Piauí,Nordeste
9,23,CE,Ceará,Nordeste


In [22]:
def load_cidades(estado_):
    url = (f'https://brasilapi.com.br/api/ibge/municipios/v1/{estado_}?providers=gov')
    req_cidades = requests.get(url)

    if req_cidades.status_code == 200:
        infos_cidades = pd.DataFrame(req_cidades.json())
        return(infos_cidades)
    else:
        alerta('3', 'cidades do estado {estado_}', 'carregamento de dados')
df_cidades=load_cidades('SP')
df_cidades.head()

,nome,codigo_ibge
0,ADAMANTINA,3500105
1,ADOLFO,3500204
2,AGUAÍ,3500303
3,ÁGUAS DA PRATA,3500402
4,ÁGUAS DE LINDÓIA,3500501


In [23]:
df_cidades = df_cidades.rename(columns={ 'nome': 'Nome', 'codigo_ibge': 'IBGE'})
df_cidades

,Nome,IBGE
0,ADAMANTINA,3500105
1,ADOLFO,3500204
2,AGUAÍ,3500303
3,ÁGUAS DA PRATA,3500402
4,ÁGUAS DE LINDÓIA,3500501
...,...,...
640,VOTORANTIM,3557006
641,VOTUPORANGA,3557105
642,ZACARIAS,3557154
643,CHAVANTES,3557204


In [24]:
def load_DDDs(DDD_):
    url = (f'https://brasilapi.com.br/api/ddd/v1/{DDD_}')
    req_DDD = requests.get(url)

    if req_DDD.status_code == 200:
        infos_DDD = pd.DataFrame(req_DDD.json())
        return(infos_DDD)
        
    else:
        alerta('3', 'informação do DDD {DDD_}', 'carregamento de dados')
df_DDD=load_DDDs('14')
df_DDD.head()

,state,cities
0,SP,IPAUÇU
1,SP,CHAVANTES
2,SP,VERA CRUZ
3,SP,URU
4,SP,UBIRAJARA


In [26]:
def salva_bd(df,nome_tabela):
    conn= sqlite3.connect('Estados_do_Brasil.db')
    df.to_sql(nome_tabela,conn,if_exists='replace',index=False)
    conn.close()
    return True